In [8]:
import pandas as pd
import yfinance as yf
from datetime import date
import datetime
import plotly.express as px
from dateutil.relativedelta import relativedelta
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px

In [9]:
dateToday = date.today()
dateToday = datetime.datetime.strptime(str(dateToday),'%Y-%m-%d').strftime('%Y-%m-%d')
fistDate = date.today()-relativedelta(years=5)
fistDate = datetime.datetime.strptime(str(fistDate),'%Y-%m-%d').strftime('%Y-%m-%d')
fixedTickers = ['ALUA.BA', 'BBAR.BA', 'BMA.BA', 'BYMA.BA', 'CEPU.BA', 'COME.BA', 'CRES.BA', 'EDN.BA', 'GGAL.BA', 'LOMA.BA', 'MIRG.BA',
                'PAMP.BA', 'SUPV.BA', 'TECO2.BA', 'TGNO4.BA', 'TGSU2.BA', 'TRAN.BA', 'TXR.BA', 'VALO.BA', 'YPFD.BA', 'M.BA']

print(fistDate, dateToday)

2018-04-23 2023-04-23


In [10]:
dataGGAL = yf.download(['GGAL.BA', 'GGAL'], start=fistDate, end=dateToday)
histCCL = dataGGAL["Adj Close"].dropna()  
histCCL = histCCL.reset_index() 
histCCL["USD"] = histCCL["GGAL.BA"]*10/histCCL["GGAL"]
histCCL = histCCL.drop(columns=['GGAL','GGAL.BA'])
display ( histCCL.tail(2) )

[*********************100%***********************]  2 of 2 completed


,Date,USD
1186,2023-04-20,436.183660
1187,2023-04-21,454.588332


In [11]:

dataPanelLider = yf.download(fixedTickers, start=fistDate, end=dateToday)
dataMerval = dataPanelLider["Adj Close"].dropna() 
dataMerval['Index'] = range(1, len(dataMerval)+1)
dataMerval = dataMerval.reset_index()
dataMerval = pd.melt(dataMerval, id_vars='Date',
                        var_name='Ticker', value_name='Close')
dataMerval.head()

[*********************100%***********************]  21 of 21 completed


,Date,Ticker,Close
0,2018-04-23,ALUA.BA,12.747694
1,2018-04-24,ALUA.BA,12.171993
2,2018-04-25,ALUA.BA,12.130873
3,2018-04-26,ALUA.BA,12.089749
4,2018-04-27,ALUA.BA,12.377603


In [12]:
dataMerval = pd.DataFrame(pd.merge(histCCL, dataMerval, how='left', on=['Date'])).dropna() 
dataMerval ['Close USD'] = dataMerval['Close']/dataMerval['USD']

dataMerval.tail()

,Date,USD,Ticker,Close,Close USD
26047,2023-04-21,454.588332,TRAN.BA,241.750000,0.531800
26048,2023-04-21,454.588332,TXR.BA,9426.000000,20.735244
26049,2023-04-21,454.588332,VALO.BA,85.300003,0.187642
26050,2023-04-21,454.588332,YPFD.BA,5185.649902,11.407354
26051,2023-04-21,454.588332,Index,1214.000000,2.670548


In [13]:
CCL = "Fx Rate (CCL): "+str(round(histCCL["USD"].iloc[-1], ndigits=1))

external_stylesheets = [dbc.themes.ZEPHYR]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.title = "Merval"
app.layout = html.Div([
                        html.H1(
                            "Historical Data - S&P Merval",
                            style={'text-align': 'left', 'background-color':'#0466C8', 'color':'#EBEBEB', "margin-bottom": "15px" ,  "padding": "10px"}
                            ),

                        html.H2(
                            CCL,
                            style={'text-align': 'left', 'color':'#9B9C9B', "margin-bottom": "15px"}
                            ),
                        
                        dbc.Row(
                            [
                                dbc.Col(
                                    html.H3(
                                    "Select a ticker:",
                                    style={'text-align': 'left', 'color':'#9B9C9B', "margin-bottom": "15px"}
                                    ),
                                    width="auto"),

                                dbc.Col(
                                    dcc.Dropdown(
                                        id="Select_Ticker",                    
                                        options=[{'label': x, 'value': x} for x in fixedTickers],
                                        multi=False,
                                        value='M.BA',
                                        style={'width': "150%", 'color': '#9B9C9B'}
                                        ),
                                    width="auto"
                                    )
                            ],
                        ),



                        dbc.Row(
                                [
                                    dbc.Col(

                                        dcc.Graph(id='lines_plot_USD',config={"displayModeBar": True}, figure={}),
                                        width="auto"),
                                
                                    dbc.Col(
                                        dcc.Graph(id='lines_plot_ARP',config={"displayModeBar": True}, figure={}),
                                        width="auto"),
                                ],
                                justify="evenly",
                                ),
                        ])
                      
                        
@app.callback(
    dash.dependencies.Output('lines_plot_USD', 'figure'),
    dash.dependencies.Output('lines_plot_ARP', 'figure'),
    [dash.dependencies.Input('Select_Ticker', 'value')])

def update_output(value):
    dff = dataMerval[dataMerval["Ticker"] == value]

    fig1 = {
                                              "data": [
                                                  {
                                                      "x": dff["Date"],
                                                      "y": dff["Close USD"],
                                                      "type": "lines",
                                                      "hovertemplate": "$%{y:.1f}<extra></extra>",
                                                  },
                                              ],
                                              "layout": {
                                                  "title": {
                                                      "text": "USD",
                                                      "x": 0.05,
                                                      "xanchor": "left",
                                                  },
                                                  "xaxis": {"fixedrange": True},
                                                  "yaxis": {"tickprefix": "$", "fixedrange": True},
                                                  "colorway": ["#007BBA"],
                                                  "width": 5,
                                                  "height": 5,
                                              },
                                          }
    fig2 = {
                                              "data": [
                                                  {
                                                      "x": dff["Date"],
                                                      "y": dff["Close"],
                                                      "type": "lines",
                                                      "hovertemplate": "$%{y:.0f}<extra></extra>",
                                                  },
                                              ],
                                              "layout": {
                                                  "title": {
                                                      "text": "ARS (Argentine Peso)",
                                                      "x": 0.05,
                                                      "xanchor": "left",
                                                  },
                                                  "xaxis": {"fixedrange": True},
                                                  "yaxis": {"tickprefix": "$", "fixedrange": True},
                                                  "colorway": ["#7CCDF4"],
                                                  "width": 5,
                                                  "height": 5,
                                              },
                                          }    

    return fig1, fig2
                              
              
app.run_server(mode='inline')